In [2]:
############ LOAD in custom packages ################
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import importlib

# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Define data directory
brighten_dir = os.path.join(project_root, 'BRIGHTEN_data')

# Add project root to sys.path
sys.path.append(project_root)

# Import and reload custom scripts
from scripts import preprocessing as pre
from scripts import visualization as vis
from scripts import feature_selection as fs
from scripts import clustering as cl
importlib.reload(pre)
importlib.reload(vis)
importlib.reload(fs)
importlib.reload(cl)

################ DEFINE column variables from data ###################
from scripts.variables import id_columns, daily_cols_v1, daily_v2_common 
from scripts.variables import phq2_cols, phq9_cols, weekly_cols, passive_cols, survey_cols


## Load in dfs scaled
df_names = ['v1_day', 'v2_day', 'v1_week', 'v2_week']

dfs_pca = {}
for name in df_names:
    dfs_pca[name] = pd.read_csv(os.path.join(brighten_dir, f'{name}_pca.csv'))

dfs_scaled = {}
for name in df_names:
    dfs_scaled[name] = pd.read_csv(os.path.join(brighten_dir, f'{name}_scaled.csv'))


In [6]:
#### Simple linear regression
from sklearn.linear_model import LinearRegression
# Step 1: Set x_cols to the various V1 PCs
y_var = 'pc_depression_phq2'
data = pd.read_csv(os.path.join(brighten_dir, 'v1_day_pca.csv'))
print(data.columns.to_list())
results = pd.DataFrame()
count=0
x_cols = [col for col in data.columns.to_list() if 'pc_' in col and col!=y_var]
y_col = [y_var]

reg = LinearRegression().fit(data[x_cols], data[y_col])
print(reg.score(data[x_cols], data[y_col]))
print(reg.coef_)
print(reg.intercept_)

['participant_id', 'num_id', 'dt', 'week', 'day', 'gender', 'age', 'aggregate_communication', 'call_count', 'call_duration', 'interaction_diversity', 'missed_interactions', 'mobility', 'mobility_radius', 'sms_count', 'sms_length', 'unreturned_calls', 'phq2_1', 'phq2_2', 'phq2_sum', 'pc_communication', 'pc_missed_communications', 'pc_mobility', 'pc_depression_phq2']
0.012852891598219895
[[0.05172084 0.09099842 0.08534029]]
[-0.00746926]


In [ ]:
######### Mixed LM Model V1 vs phq2_sum #########
###### V1 PCs vs phq2_sum #########
from statsmodels.regression.mixed_linear_model import MixedLM
import statsmodels.api as sm
import statsmodels.formula.api as smf

ignore_cols = ['num_id', 'dt', 'week', 'day']
y_var = 'phq9_sum'
data = pd.read_csv(os.path.join(brighten_dir, 'v1_week.csv'))
results = pd.DataFrame()
count = 0
x_cols = [col for col in data.columns.to_list() if 'phq9' not in col and col not in ignore_cols]
data_full = data[x_cols + [y_var, 'num_id']].copy().dropna()
data_full["num_id"] = data_full["num_id"].astype("category")
display(data_full)
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

model = smf.mixedlm(f"{y_var} ~ aggregate_communication + call_count + call_duration + interaction_diversity + missed_interactions + mobility + mobility_radius + sms_count + sms_length + unreturned_calls + phq2_1 + phq2_2 + phq2_sum", 
                    data=data_full, 
                    groups=data_full["num_id"])
result = model.fit()
print(f'Relationship to y_col {y_var}, Nested Linear Regression on "num_id"')
print(result.summary())



,aggregate_communication,call_count,call_duration,interaction_diversity,missed_interactions,mobility,mobility_radius,sms_count,sms_length,unreturned_calls,phq2_1,phq2_2,phq2_sum,phq9_sum,participant_id
1184,38.0,7.0,5412.0,7.0,1.0,9.334,5.597,31.0,3336.0,0.0,3.000000,2.000000,5.000000,9.0,BLUE-00062
1260,26.0,3.0,158.0,2.0,0.0,3.123,33.944,23.0,978.0,0.0,2.000000,1.000000,3.000000,10.0,BLUE-00063
1574,123.0,10.0,9902.0,8.0,5.0,1.536,3.223,113.0,6698.0,2.0,2.000000,2.000000,4.000000,8.0,BLUE-00066
1588,118.0,10.0,5551.0,10.0,6.0,0.990,11.427,108.0,5457.0,6.0,2.000000,2.000000,4.000000,8.0,BLUE-00066
1602,104.0,7.0,772.0,7.0,2.0,0.338,0.017,97.0,5701.0,0.0,2.000000,2.000000,4.000000,12.0,BLUE-00066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79687,9.0,2.0,126.0,2.0,0.0,0.444,11.349,7.0,355.0,0.0,1.000000,1.000000,2.000000,3.0,YELLOW-00260
79758,259.0,3.0,4563.0,8.0,1.0,2.320,3.731,256.0,17701.0,0.0,2.833333,2.833333,5.666667,6.0,YELLOW-00262
79765,130.0,3.0,3557.0,11.0,8.0,1.177,2.243,127.0,6712.0,8.0,2.000000,3.000000,5.000000,10.0,YELLOW-00262
79782,61.0,26.0,14992.0,16.0,1.0,0.845,0.124,35.0,951.0,0.0,5.000000,5.000000,10.000000,12.0,YELLOW-00263


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(


LinAlgError: Singular matrix